## Hi love, I want you to meet VerseMonger 1.0 

##### The best songs ever written were those that came from the heart. That made you feel.

##### But a clever songwriter consciously uses poetic devices, meters and tricks to enchance the effect of a song. 



I simply hope this tool helps you be more conscious about your writing.

![alt text](https://cdni.rbth.com/rbthmedia/images/2018.08/original/5b7c34d015e9f93b263a3bca.jpg)


Alright, here's what you need to know:

You must open this in playground mode. If it asks you to save your own copy, you can do that too.

1.   Each of these boxes with code/text written in them is called a cell.
2.   You have to run a cell for it to give you an output.
3.   You run a selected cell by pressing [shift] + [enter]
4.   Incase the cell isn't selected, you might have to click on it first. 
5.   You'll see the output of a cell right below it.
6.   You can always clear an output by clicking on the cross. You'll find the cross on the top left corner of the output area.
7.  Don't worry about the code, I have a copy.
8.  You can always restart your code to start fresh. You don't need to run everything again though. I'll show you where to start.
9.  You must ask for changes in this code to suit your requirements.
10. Incase you want to scroll, make sure you take your mouse to the scroll bar (extreme right). Else you'll end up scrolling through the output cells.
11. Whatever sections you're done running, you can fold them. So you don't have to scroll through them. You'll see an arrow right next to the section headers.Click on it to fold them.
12. Mainly, you'll want to fold-> **Setting up** and **Processing the song**
13. If you find the kittens annoying, just go to (top right corner) settings -> Miscellaneous -> enable kitty mode
 



# Setting up.

You must click on the cell below and press [shift] + [enter]

And then scroll down obviously.

In [ ]:
!pip install  pronouncing
!pip install phyme

import pandas as pd
import string
import nltk
import random

nltk.download('cmudict')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import word_tokenize 
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.corpus import cmudict

from itertools import product
import pronouncing as pro
from Phyme import Phyme
import itertools

cmu = cmudict.dict()
ph = Phyme()

def count_syllables(word):
    lower_word = word.lower()
    if lower_word in cmu:
        return max([len([y for y in x if y[-1] in string.digits])
                    for x in cmu[lower_word]])


def clean_data(df):
    df['lines'] = df['original_lines'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    df['lines'] = df['lines'].str.replace('[^\w\s]','')
    df['lines'] = df['lines'].str.lower()
    return df


def get_lines(song):
    lines = song.splitlines()
    return lines


def get_words(lines):
    song_words = []
    for line in lines:
        temp_words = nltk.word_tokenize(line)
        line_words= [word for word in temp_words if word.isalnum()]
        song_words.append(line_words)
    return song_words


def count_song_syllable(song_words):
    syllable_count = []
    for song_line in song_words:
        if song_line == []:
            syllable_count.append('x')
        else:
            syllable = []
            for word in song_line:
                syllable.append(count_syllables(word))
            syllable_count.append(syllable)
    return syllable_count


def get_words_syllables(song_words, syllable):
    for i, line in enumerate(syllable):
        if line != 'x':
            for j, word in enumerate(line):
                if word == None:
                    print(song_words[i][j])
                    print("Rewrite the word: ")
                    song_words[i][j] = input()
                    print("Enter the approximate syllable count: ")
                    syllable[i][j] = int(input())

    return song_words, syllable


def display_line_count(syllable):
    avg = 0
    for ctr, s in enumerate(syllable):
        if s != 'x':
            print("Line ", ctr, " syllable count: ", sum(s))
            avg = avg + sum(s)
    avg = int(avg/(ctr+1))
    print("\nAverage syllable count: ", avg)
    return avg


def get_pos(song):
  text = word_tokenize(song.lower())
  so = nltk.pos_tag(text)
  pos = pd.DataFrame(so, columns = ['word', 'pos'])
  return pos


def count_stats(df):
  stanza_no = 1
  line_no = 0
  word_count = 0
  char_count = 0

  df['stanza']  = 0
  df['line']  = 0
  df['word_count']  = 0
  df['syllable_count'] = 0
  df['char_count'] = 0
  
  for i, j in enumerate(df['syllable']):
    if j == 'x':
      stanza_no += 1
    else:
      line_no += 1
      df['stanza'][i] = stanza_no
      df['line'][i] = line_no
      df['word_count'][i] = len(df['words'][i])
      df['syllable_count'][i] = sum(df['syllable'][i])
      for element in df['words'][i]:
        df['char_count'][i] += len(element)

  return df


def stop_topic_words(df):
  stop = stopwords.words('english')
  df['stopwords'] = df['lines'].apply(lambda x: [x for x in x.split() if x.lower() in stop])
  df['topic'] = df['lines'].apply(lambda x: [x for x in x.split() if x.lower() not in stop])
  return df


def get_stress(phones):
  syll = []
  stress = []
  syl = ''
  for c, l in enumerate(phones[0].split()):
    syl = syl + l
    if syl[-1].isnumeric():
      syll.append(syl[:-1])
      stress.append(syl[-1])
      syl = ''
    if c == len(phones[0].split())-1:
      syll[-1] = syll[-1] + syl
  return syll, stress


def get_stress_syllables(df):
  df['syllables_text'] = 0
  df['stress'] = 0
  for i, line in enumerate(df['words']):
    line_syllables = []
    line_stress = []
    for word in line:
      phones = pro.phones_for_word(word)
      if len(phones) > 1:
        phones = [phones[0]]
      if phones == []:
        syllables, stress = '???', '?'
      else:
        syllables, stress = get_stress(phones)
      line_syllables.append(syllables)
      line_stress.append(stress)
    df['syllables_text'][i] = line_syllables
    df['stress'][i] = line_stress
  return df


def get_synonyms(word):
  synonyms = [] 
  antonyms = []  
  for syn in wn.synsets(word): 
      for l in syn.lemmas(): 
          synonyms.append(l.name()) 
          if l.antonyms(): 
              antonyms.append(l.antonyms()[0].name())             
  return list(set(synonyms))


def get_all_synonyms(df):
  df['synonyms'] = 0
  for no, line in zip(df.index, df['words']):
    syn = []
    for w in line:
      word_syn = get_synonyms(w)
      #print("The synonyms of ", w, word_syn)
      syn.append(word_syn)
    df['synonyms'][no] = syn
  return df


def get_all_rhymes(df):
  df['perfect_rhyme'] = 0
  #df['partner_rhyme'] = 0
  df['assonance_rhyme'] = 0
  df['consonant_rhyme'] = 0
  #df['substitution_rhyme'] = 0
  perfect_rhyme = []
  #partner_rhyme = []
  assonance_rhyme = []
  consonant_rhyme = []
  #substitution_rhyme = []

  for c, top in zip(df.index, df['words']):
    perf_rhyme = []
    #part_rhyme = []
    asso_rhyme = []
    cons_rhyme = []
    #sub_rhyme = []
    for t in top:
      
      try:
        perf_rhy = ph.get_perfect_rhymes(t)
        #part_rhy = ph.get_partner_rhymes(t)
        asso_rhy = ph.get_assonance_rhymes(t)
        cons_rhy = ph.get_consonant_rhymes(t)
        #sub_rhy = ph.get_substitution_rhymes(t) 
      
      except:
        perf_rhy = None
        #part_rhy = None
        asso_rhy = None
        cons_rhy = None
        #sub_rhy = None  
      
      perf_rhyme.append(perf_rhy)
      #part_rhyme.append(part_rhy)
      asso_rhyme.append(asso_rhy)
      cons_rhyme.append(cons_rhy)
      #sub_rhyme.append(sub_rhy)
  
    perfect_rhyme.append(perf_rhyme)
    #partner_rhyme.append(part_rhyme)
    assonance_rhyme.append(asso_rhyme)
    consonant_rhyme.append(cons_rhyme)
    #substitution_rhyme.append(sub_rhyme)

  df['perfect_rhyme'] = perfect_rhyme
  #df['partner_rhyme'] = partner_rhyme
  df['assonance_rhyme'] = assonance_rhyme
  df['consonant_rhyme'] = consonant_rhyme
  #df['substitution_rhyme'] = substitution_rhyme
  return df


def get_similarity(wordx, wordy):
  sem1, sem2 = wn.synsets(wordx), wn.synsets(wordy)

  maxscore = 0
  for i,j in list(product(*[sem1,sem2])):
    score = i.wup_similarity(j) # Wu-Palmer Similarity
    #print(i, j, score)
    if score != None:
      maxscore = score if maxscore < score else maxscore
  return maxscore


def insert_rym(rymzone, word1, word2, rhymer, score, syllb):
  if word1 not in rymzone.keys():
    rymzone[word1] = dict()
  if word2 not in rymzone[word1].keys():
    rymzone[word1][word2] = dict()
  rymzone[word1][word2]['rhymer'] = rhymer
  rymzone[word1][word2]['score'] = score
  rymzone[word1][word2]['syllb'] = syllb
  return rymzone


def get_relevat_rhymes(word1, dt, rymzone):
  for b, rows in zip(dt.index, dt['perfect_rhyme']):
    for c, dic in enumerate(rows):
      for syllb in dic:
        for word in dic[syllb]:
          word2 = word.split("(")[0]
          score = get_similarity(word2, word1)
          if score > 0.7:
            #print(word1, word2, dt['words'][b][c], score)
            rymzone = insert_rym(rymzone, word1, word2, dt['words'][b][c], score, syllb)
  return rymzone


def get_rhymezone(dt):
  rymzone = dict()
  for p, ro in zip(dt.index, dt['topic']):
    for q, word1 in enumerate(ro):
      rymzone = get_relevat_rhymes(word1, dt, rymzone)
  return rymzone
    

def create_rhymezone(df):
  rhymezone = dict()
  stanza_len = df['stanza'].max()
  for sl in range(1,stanza_len+1):
    dt = df.loc[df.stanza == sl]
    rhymezone[sl] = get_rhymezone(dt)
  return rhymezone




def insert_asso(asozone, word1, word2, rhymer, score, syllb):
  if word1 not in asozone.keys():
    asozone[word1] = dict()
  if word2 not in asozone[word1].keys():
    asozone[word1][word2] = dict()
  asozone[word1][word2]['rhymer'] = rhymer
  asozone[word1][word2]['score'] = score
  asozone[word1][word2]['syllb'] = syllb
  return asozone


def get_relevat_asso(word1, dt, asozone):
  for b, rows in zip(dt.index, dt['assonance_rhyme']):
    for c, dic in enumerate(rows):
      for syllb in dic:
        for word in dic[syllb]:
          word2 = word.split("(")[0]
          score = get_similarity(word2, word1)
          if score > 0.85:
            #print(word1, word2, dt['words'][b][c], score)
            if len(word2) > 2:
              asozone = insert_rym(asozone, word1, word2, dt['words'][b][c], score, syllb)
  return asozone


def get_assozone(dt):
  asozone = dict()
  for p, ro in zip(dt.index, dt['topic']):
    for q, word1 in enumerate(ro):
      asozone = get_relevat_rhymes(word1, dt, asozone)
  return asozone
    

def create_assonance(df):
  assonance = dict()
  stanza_len = df['stanza'].max()
  for sl in range(1,stanza_len+1):
    dt = df.loc[df.stanza == sl]
    assonance[sl] = get_assozone(dt)
  return assonance


def insert_conso(consozone, word1, word2, rhymer, score, syllb):
  if word1 not in consozone.keys():
    consozone[word1] = dict()
  if word2 not in consozone[word1].keys():
    consozone[word1][word2] = dict()
  consozone[word1][word2]['rhymer'] = rhymer
  consozone[word1][word2]['score'] = score
  consozone[word1][word2]['syllb'] = syllb
  return consozone


def get_relevat_conso(word1, dt, consozone):
  for b, rows in zip(dt.index, dt['consonant_rhyme']):
    for c, dic in enumerate(rows):
      for syllb in dic:
        for word in dic[syllb]:
          word2 = word.split("(")[0]
          score = get_similarity(word2, word1)
          if score > 0.8:
            #print(word1, word2, dt['words'][b][c], score)
            if len(word2) > 2:
              consozone = insert_rym(consozone, word1, word2, dt['words'][b][c], score, syllb)
  return consozone


def get_consozone(dt):
  consozone = dict()
  for p, ro in zip(dt.index, dt['topic']):
    for q, word1 in enumerate(ro):
      consozone = get_relevat_rhymes(word1, dt, consozone)
  return consozone
    

def create_consonant(df):
  consonant = dict()
  stanza_len = df['stanza'].max()
  for sl in range(1,stanza_len+1):
    dt = df.loc[df.stanza == sl]
    consonant[sl] = get_consozone(dt)
  return consonant


def find_alliteration(line, smallest_seq):  
    seq = 1
    words = line.split()
    letter = words[0][0]
    for word in words[1:]:
        if word[0] == letter:
            seq += 1
            if seq >= smallest_seq:
                return line
        else:
            letter = word[0]
            seq = 1
    return None


def construct(song):
  if song[0] == '\n':
    song = song[1:]
  df = pd.DataFrame() 
  df['original_lines']  = get_lines(song)
  df = clean_data(df)
  df['words'] = get_words(df['lines'])
  df['syllable'] = count_song_syllable(df['words'])
  df['words'], df['syllable'] = get_words_syllables(df['words'], df['syllable'])
  return df


def prep_stats(df):
  df = count_stats(df)
  df = stop_topic_words(df)
  df = get_stress_syllables(df)
  df = get_all_synonyms(df)
  df = get_all_rhymes(df)
  rhymezone = create_rhymezone(df)
  consonant = create_consonant(df)
  assonance = create_assonance(df)
  return df, rhymezone, consonant, assonance


def say_hi():
  boo_rhyme = ph.get_assonance_rhymes('boo')[1]
  boo_rhyme = random.choice(boo_rhyme)
  great = get_synonyms('fantastic') + get_synonyms('excellent') 
  great = random.choice(great)
  boo = 'baby' + boo_rhyme.split('(')[0]
  print(boo, "this song is", great)


def song_stats(df):
  print("")
  print("You wrote", df['stanza'].max(), "verses,", df['line'].max(), "lines and", df['word_count'].sum(), "words.")
  print("")
  line_avg_syllb = df['syllable_count'].sum() / df['line'].max()
  line_avg_word = df['word_count'].sum() / df['line'].max()
  print("On an average, you used about", int(round(line_avg_syllb)), "syllables and", int(round(line_avg_word)), "words per line.")

  word_avg_char = df['char_count'].sum() / df['word_count'].sum()
  word_avg_syllb =  df['syllable_count'].sum() / df['word_count'].sum()
  print("On an average, you used about", int(round(word_avg_char)), "charachters and", int(round(word_avg_syllb)), "syllable/s per word.")


  pos = get_pos(song)
  print("\n\nPart of Speech:")
  nouns = pos.loc[pos['pos'].isin(['NN', 'NNS', 'NNP', 'NNPS'])]
  top_nouns = nouns.word.value_counts()[:10]
  print("You used like", len(nouns), "Nouns. The most frequent ones being: \n", list(zip(top_nouns.index, top_nouns)),'\n')

  adjective = pos.loc[pos['pos'].isin(['JJ', 'JJR', 'JJS'])]
  top_adjective = adjective.word.value_counts()[:10]
  print("You used like", len(adjective), "Adjectives. The most frequent ones being: \n", list(zip(top_adjective.index, top_adjective)),'\n')

  verb = pos.loc[pos['pos'].isin(['VB', 'VBD', 'VBG','VBN', 'VBP', 'VBZ'])]
  top_verb = verb.word.value_counts()[:10]
  print("You used like", len(verb), "Verbs. The most frequent ones being: \n", list(zip(top_verb.index, top_verb)),'\n')

  sw = list(itertools.chain(*list(df['stopwords'])))
  sw = pd.Series(sw)
  top_sw = sw.value_counts()[:10]
  print("And you used like", sw.nunique(), "unique stopwords that repeated a total of", len(sw), "times. The most frequent ones being: \n", list(zip(top_sw.index, top_sw)),'\n')

  print("\n\nYour alliterative lines: \n")
  for line in df['lines']:
    if line !='':
      alliteration = find_alliteration(line, 2)
      if alliteration != None:
        print(alliteration)


def stats_per_verse(df):
  print("Lets look at every verse in more detail.\n\n")
  for i in range(1, df['stanza'].max() + 1):
    print("Verse:", i, "\n")
    song_t = ''
    
    dt = df[df.stanza == i]
    for l in dt.lines:
      print(l)
      song_t = song_t + l + " "
    
    print("\nSome stats for my baby:")
    print("-> This verse has", len(dt['line']), "lines and", dt['word_count'].sum(), "words.")
    
    line_avg_syllb = dt['syllable_count'].sum() / len(dt['line'])
    line_avg_word = dt['word_count'].sum() / len(dt['line'])
    print("-> On an average, this verse has about", round(line_avg_syllb, 2), "syllables and", round(line_avg_word, 2), "words per line.")
    
    word_avg_char = dt['char_count'].sum() / dt['word_count'].sum()
    word_avg_syllb =  dt['syllable_count'].sum() / dt['word_count'].sum()
    print("-> On an average, this verse has about", round(word_avg_char, 2), "charachters and", round(word_avg_syllb, 2), "syllable/s per word.")

    print("\nPart of Speech:")
    pos = get_pos(song_t)

    nouns = pos.loc[pos['pos'].isin(['NN', 'NNS', 'NNP', 'NNPS'])]
    top_nouns = nouns.word.value_counts()[:10]
    print("->You used like", len(nouns), "Nouns. The most frequent ones being: \n", list(zip(top_nouns.index, top_nouns)),'\n')

    adjective = pos.loc[pos['pos'].isin(['JJ', 'JJR', 'JJS'])]
    top_adjective = adjective.word.value_counts()[:10]
    print("->You used like", len(adjective), "Adjectives. The most frequent ones being: \n", list(zip(top_adjective.index, top_adjective)),'\n')

    verb = pos.loc[pos['pos'].isin(['VB', 'VBD', 'VBG','VBN', 'VBP', 'VBZ'])]
    top_verb = verb.word.value_counts()[:10]
    print("->You used like", len(verb), "Verbs. The most frequent ones being: \n", list(zip(top_verb.index, top_verb)),'\n')

    sw = list(itertools.chain(*list(dt['stopwords'])))
    sw = pd.Series(sw)
    top_sw = sw.value_counts()[:10]
    print("->And you used like", sw.nunique(), "unique stopwords that repeated a total of", len(sw), "times. The most frequent ones being: \n", list(zip(top_sw.index, top_sw)),'\n')

    print("\n\n")
    ans = input()


def stats_per_line(df):
  print("Alright baby, now lets check each line out.\n\n")

  for i in df.index:
    if df['stanza'][i] != 0:
      print("Verse", df['stanza'][i], "Line", df['line'][i])
      print(df['lines'][i])
      print("-> Word Count:", df['word_count'][i])
      print("-> Character Count:", df['char_count'][i])
      dt = df[df.stanza == df.stanza[i]]
      avg_syl = round(dt['syllable_count'].sum()/len(dt['syllable_count']))
      syllable_need = avg_syl - dt['syllable_count'][i]
      print("-> Syllable Count:", df['syllable_count'][i])
      print("-> Avg Syllable Count of the verse:", avg_syl, " |  Syllable need: ", syllable_need)
      
      word_avg_char = df['char_count'][i] / df['word_count'][i]
      word_avg_syllb =  df['syllable_count'][i] / df['word_count'][i]
      print("-> On an average, this line has roughly", round(word_avg_char, 2), "charachters and", round(word_avg_syllb, 2), "syllable/s per word.")
      #print("\n")
      print("-> List of syllables:", df['syllables_text'][i])
      print("-> Stress of each syllable:", df['stress'][i])
      print("\n\n")
    ans = input()


def perfect_recco(df, rhymezone):
  for i in range(1, df['stanza'].max() + 1):
    print("Verse:", i, "\n")
    song_t = ''
    
    dt = df[df.stanza == i]
    avg_syl = round(dt['syllable_count'].sum()/len(dt['syllable_count']))
    print("The average syllable count is: ", avg_syl,"\n")

    print()
    for m in dt.index:
      
      for c, l in zip(dt.index, dt.lines):
        print("Line", dt.line[c], "|", l, " | Syllable count:", dt.syllable_count[c])
        song_t = song_t + l + " "
      
      print()
      print("-> In line", dt.line[m])
      syllable_need = avg_syl - dt['syllable_count'][m]
      print("-> You could maybe adjust your syllable count by", syllable_need)
      print("-> These are some rhyming synonyms\n")

      for ctr, n in enumerate(dt.topic[m]):
        try:
          if rhymezone[i][n]:
            print("- -> For the word:", n)
            for o in rhymezone[i][n]:
              if n!=o:
                print("- - -> ", o, "| syllables:",rhymezone[i][n][o]['syllb'],"| rhymes with:", rhymezone[i][n][o]['rhymer']) 
            print("")
        except:
          pass
      print("\n")
      input()

    print("\n\n")


def asso_recco(df, assonance):
  for i in range(1, df['stanza'].max() + 1):
    print("Verse:", i, "\n")
    song_t = ''
    
    dt = df[df.stanza == i]
    avg_syl = round(dt['syllable_count'].sum()/len(dt['syllable_count']))
    print("The average syllable count is: ", avg_syl,"\n")

    print()
    for m in dt.index:
      
      for c, l in zip(dt.index, dt.lines):
        print("Line", dt.line[c], "|", l, " | Syllable count:", dt.syllable_count[c])
        song_t = song_t + l + " "
      
      print()
      print("-> In line", dt.line[m])
      syllable_need = avg_syl - dt['syllable_count'][m]
      print("-> You could maybe adjust your syllable count by", syllable_need)
      print("-> These are some rhyming synonyms\n")

      for ctr, n in enumerate(dt.topic[m]):
        try:
          if assonance[i][n]:
            print("- -> For the word:", n)
            for o in assonance[i][n]:
              if n!=o:
                print("- - -> ", o, "| syllables:",assonance[i][n][o]['syllb'],"| rhymes with:", assonance[i][n][o]['rhymer']) 
            print("")
        except:
          pass
      print("\n")
      input()

    print("\n\n")


def conso_recco(df, consonant):
  for i in range(1, df['stanza'].max() + 1):
    print("Verse:", i, "\n")
    song_t = ''
    
    dt = df[df.stanza == i]
    avg_syl = round(dt['syllable_count'].sum()/len(dt['syllable_count']))
    print("The average syllable count is: ", avg_syl,"\n")

    print()
    for m in dt.index:
      
      for c, l in zip(dt.index, dt.lines):
        print("Line", dt.line[c], "|", l, " | Syllable count:", dt.syllable_count[c])
        song_t = song_t + l + " "
      
      print()
      print("-> In line", dt.line[m])
      syllable_need = avg_syl - dt['syllable_count'][m]
      print("-> You could maybe adjust your syllable count by", syllable_need)
      print("-> These are some rhyming synonyms\n")

      for ctr, n in enumerate(dt.topic[m]):
        try:
          if consonant[i][n]:
            print("- -> For the word:", n)
            for o in consonant[i][n]:
              if n!=o:
                print("- - -> ", o, "| syllables:",consonant[i][n][o]['syllb'],"| rhymes with:", consonant[i][n][o]['rhymer']) 
            print("")
        except:
          pass
      print("\n")
      input()

    print("\n\n")
    

def display_song(df):
  stanza = 0
  for i in df.index:
    if df.stanza[i] != 0:
      if stanza != df.stanza[i]:
        dt = df[df.stanza == df.stanza[i]]
        avg_syl = round(dt['syllable_count'].sum()/len(dt['syllable_count']), 2)
        print("\nVerse", df.stanza[i]," | Avg Syllables", avg_syl)
        stanza = df.stanza[i]
      print("Line", df.line[i],"-> ", df['lines'][i], " | Syllables:", df.syllable_count[i])

def syn_tool(df):
  ans = input("Enter line number or word:")

  if ans.isnumeric():
    no = int(ans)
    print("\nLine", no, "->", df['lines'][no],"\n\n")
    for ctr, word in enumerate(df['words'][no]):
      print("Synonyms for the word ->", word)
      print("- -> ", df['synonyms'][no][ctr])
      print("\n")
  elif ans.isalpha():
    word = ans.lower()
    print("\nWord:", word)
    for cc, row in enumerate(df['words']):
      if word in row:
        for ctr, w in enumerate(row):
          if word == w:
            print("Synonyms for the word ->", word)
            print("- -> ", df['synonyms'][cc][ctr])
  else:
    print("Wtf, this isn't a number or a proper word. Check your input again.")


def tool_perf_rhyme(df):
  ans = input("Enter line number or word:")

  if ans.isnumeric():
    no = int(ans)
    print("\nLine", no, "->", df['lines'][no],"\n\n")
    for ctr, word in enumerate(df['words'][no]):
      print("Perfect rhymes for the word ->", word)
      for x in df['perfect_rhyme'][no][ctr].keys():
        print("Syllables count - ->", x)
        print("- -> ", df['perfect_rhyme'][no][ctr][x])
      print("\n")
  elif ans.isalpha():
    word = ans.lower()
    print("\nWord:", word)
    cons_rhy = ph.get_perfect_rhymes(word)
    print("Perfect rhymes for the word ->", word)
    for x in cons_rhy.keys():
      print("Syllables count - ->", x)
      print("- -> ", cons_rhy[x])
  else:
    print("Wtf, this isn't a number or a proper word. Check your input again.")


def tool_asso(df):
  ans = input("Enter line number or word:")

  if ans.isnumeric():
    no = int(ans)
    print("\nLine", no, "->", df['lines'][no],"\n\n")
    for ctr, word in enumerate(df['words'][no]):
      print("Assonance for the word ->", word)
      for x in df['assonance_rhyme'][no][ctr].keys():
        print("Syllables count - ->", x)
        print("- -> ", df['assonance_rhyme'][no][ctr][x])
      print("\n")
  elif ans.isalpha():
    word = ans.lower()
    print("\nWord:", word)
    cons_rhy = ph.get_assonance_rhymes(word)
    print("Assonance for the word ->", word)
    for x in cons_rhy.keys():
      print("Syllables count - ->", x)
      print("- -> ", cons_rhy[x])
  else:
    print("Wtf, this isn't a number or a proper word. Check your input again.")


def tool_conso(df):
  ans = input("Enter line number or word:")

  if ans.isnumeric():
    no = int(ans)
    print("\nLine", no, "->", df['lines'][no],"\n\n")
    for ctr, word in enumerate(df['words'][no]):
      print("Consonants for the word ->", word)
      for x in df['consonant_rhyme'][no][ctr].keys():
        print("Syllables count - ->", x)
        print("- -> ", df['consonant_rhyme'][no][ctr][x])
      print("\n")
  elif ans.isalpha():
    word = ans.lower()
    print("\nWord:", word)
    cons_rhy = ph.get_consonant_rhymes(word)
    print("Consonants for the word ->", word)
    for x in cons_rhy.keys():
      print("Syllables count - ->", x)
      print("- -> ", cons_rhy[x])
  else:
    print("Wtf, this isn't a number or a proper word. Check your input again.")


def tool_stress_syll():
  word = input("Enter a word: ")
  phones = pro.phones_for_word(word)
  syll, stress = get_stress(phones)
  print(syll)
  print(stress)
  



     |████████████████████████████████| 942kB 3.9MB/s 
  Created wheel for pronouncing: filename=pronouncing-0.2.0-py2.py3-none-any.whl size=6223 sha256=066e811d50f52de85afe78a9a8bff005024d679d2a33aa93face139196129c14
  Stored in directory: /root/.cache/pip/wheels/81/fd/e8/fb1a226f707c7e20dbed4c43f81b819d279ffd3b0e2f06ee13
Successfully built pronouncing
     |████████████████████████████████| 1.3MB 2.7MB/s 
  Created wheel for phyme: filename=Phyme-0.0.9-cp36-none-any.whl size=1379058 sha256=156426801c1e78a86bf1f66dfd15ad15908626432e7cab8b298a471105ff661e
  Stored in directory: /root/.cache/pip/wheels/68/79/ca/e58a1f9509af3537f34c9c98ab6dfcf56f1b7cd40788c9a46d
Successfully built phyme
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpo

#### Shutting Outputs

Do you see that small box with an arrow? It should be on the top left corner of this output. It'll be under a [1]^

When you hover on it, it'll turn into a cross. Click on it.

We don't want to waste our life scrolling this dumb output.Close anything that means nothing to you.

# Songs

This is where we'll keep our songs. Everytime you write a song, you can add it here. Just give the song a name. We'll use the name to run the song below.

Make sure you insert the text within three inverted commas, like I have.

In [ ]:

song9 = """
insert new song here
and keep a track of the names
Like the name of this song is song9
"""


song8 = """
The frogs they all sing
The cars are nowhere
The bells they all ring 
For the death of the fair
 
But please let me out
So the world I can see
I swear I’m devout
Just please let me leave
 
I heard the mockingbird
And he was mocking me
So free and happy, and so absurd
Flying cross the Arabian Sea
 
I see them in beds
In the depth of my dreams
A fate worse than lead
It is what it seems
 
But please let me out
The birds are so free
I swear I will shout
Just please let me leave
 
I have forgotten how
The wind mingles with noise
The corner of my bedroom floor
Is full with all my stupid toys
 
Oh oh oh 
Let me out 
Oh oh oh 
I will shout
"""

song7 = """
Oh sweet sorrow
That lives in this burrow
Let me be so free and fine
Let me bask and furrow

The sun it rises 
Each day with no surprises
The calling of the little bird
So shrill in this crisis

Oh my love I heard you there 
Over by the giant stairs
I walked straight down with sweat on my brow
To find your lonely empty chair

The winds all weep for you rest 
Your head upon my feeble chest
I pray all day for you to come
Take me dancing, rid my glum 

Oh I need you more
Than I ever did before
I’m breathing oh my love.
I’m breathing

I forgot to tell you how
We may be apart right now
But when you call my name I’ll come
On the wind, between your thumbs  

Oh sweet sorrow
That lives in my furrows 
Let me be so free and fine
Let me leave this burrow
"""

song6 = """
The grave digger and his bodies
Find no rest, no room for folly 
Into the night
Fresh out of life
The ghosts all loom
Dancing ‘neath the moon 
He says a prayer
And does prepare
That final bed
For the wretched dead
Underneath the moon 
Weeping moon.
 
The gravedigger hears them calling
Every day, the freshly fallen
Not a single pyre
Will be lit on fire
He knows the tune.
Death is for the doomed.
Lying on the right
For mecca
When the morning light
Comes to flicker
It’ll be his turn soon.
All too soon.
"""

song5 = """
You broke me down
You built me new
You left me, cold, till I came to
 
Oh boy, you cried, you bled, and you said
That’ll you’ll always be there for me, no
 
I kept you safe
I kept you cool
I strung along, sick, like a fool
 
Oh boy, it’s plain, you left me, in the rain 
But you still make me better no
So I can’t pack my things and go
 
You’re a living, you’re a breathing, wonder
Full of hope and awe that you did plunder
From my little bones 
From my feral toes
You’re my tale of woe 
 
Oh boy, you cried, you bled and you said
That you’ll always be there for me no 
Oh boy, it’s plain, you left me, in the rain 
But you still make me better no
So I can’t pack my things and go
 
You still want me, you still need me, lover
For all my faults, you want me in the shower
For my golden skin
For all worldly sin
How can I win? 
 
You broke me down
You built me new
You have so much pride left in you 
 
Oh boy, it’s true, you left me, I love you
But you can never see it for me no
Guess I’d have to kill for you to show
"""


song4_3 = """
He makes her happy, he thinks
House on the beach, a life on the brink
The flowers all bloom
When they take the room 
But she’s had some time to think
Looks softly at him, over his drink
‘Oh my love’ she breathes in confession
Of wandering eye, chasing indiscretion
For she’s been lost, lost and daydreaming
Of other boys, all nasty and gleaming

But the flowers all bloom
When they take the room
He sighs she cries
She drips from her thighs
A quiet desire
Three lovers, she lies

He spills the last of his drink
Blood in his eyes, his gut starts to sink
For he built this house
A house for his spouse
And his two little children to come
That he’d never belt, while on the rum 
‘Oh sweet whore’ he screamed with no couther
Like he was raised, by his wayward mother
His eyes grew black, his rage beyond reason
For her weary crimes, long out of season 

The flowers all wilt
In the house that he built
He blights he fights 
She slips out at night
A quiet desire
Grows louder with fire

She’s dressed in black on the run
Her skirt all unfurled, her hair all undone
She walks in the room
Where wanton men loom
For how oh how, can she fight the moon
The waves of despair 
That were coming from there
Washed away faith all too soon 
‘Oh strange man’ she cried in her pleasure 
Her clothes ripped off, for their randy leisure
She was pierced and dragged, she was bold and she was beaten
For after all, Its been ages since she’s eaten

The flowers all bloom
When she takes the room
She gasps she runs
She has to beat the sun
To go back to love
That cannot be undone.
"""


song4_2 = """
‘I make her happy’, he thinks 
Blood in his eyes, his gut starts to sink
For he built a house
A house for his spouse
And the two little children to come
That he’d never hit, while on the rum 
‘Oh sweet whore’ he screamed with no couther
Like he was raised, by his wayward mother
‘The rules of love, truer than a stricture’
How dare she come, and wreck his perfect picture 
 
But the flowers all bloom
When they take the room
He sighs she cries
She drips from her thighs
A quiet desire
Three lovers, she lies
 
On the porch, far past the sun
Skirt all unfurled, her hair all undone. 
The flowers did bloom
When they took the room 
But how oh how, could she fight the moon 
The waves of despair
That we’re coming from there
Washed away hope all too soon
‘Oh my love, I’ll love you forever,
But I must leave and go to another
Her bags were packed, she left him asunder
To go her way, on to hopeless wonders 
 
But the flowers all bloom
When they take the room
He sighs, she cries
She drips from her thighs 
A quiet desire
Three lovers, she lies
"""

song4_1 = """
He makes her happy, he thinks
House on the beach, a life on the brink
The flowers all bloom
When they take the room 
But she’s had some time to think
Looks softly at him, over his drink
‘Oh my love’ she cries in confession
Of wandering eye, chasing indiscretion
For she’s been lost, lost and daydreaming
Of other boys, all nasty and gleaming
 
But the flowers all bloom
When they take the room
He sighs she cries
She drips from her thighs
A quiet desire
Three lovers, she lies
"""


song3 = """
I don’t know
Why he’s so cold
I only took his knife
And slit him whole
Then I bit his lungs
On the quiet floor
The air was seeping
His skin was weeping
The sky was fire
The ground was snow
Then he sat there thawing in his mould
Still as ice
Not a flinch in sight
But a quiver in my bones.
A quiver in my bones.

There I sat
All wet and soaked
Save lips too dry
My stomach broke

Oooh sweet solace
Come to me

Then I ran quick
Straight to Carter’s
Where all my friends
Laughed with their partners
My blood/head was swollen
But my skin was golden
In the waning sun
The kids were on the run
So I smoked and sang
In to the night
The air warm
The sky was bright
And all at once
No wrongs no right

I finally felt home
I finally felt home

There was a bar
Then there was a car
I’m pretty sure my friend
He hit some people
On a night like this
By Hill road’s steeple
And Rosie there
With her pretty hair
Could give a scare
With a butcher’s flare

Listen, pass the joint

After all,
it’s a bloody business
All the boys
Cheating on their missus
All the girls
Plotting retribution
With poison pills
In distribution

They’ve got blood of their own
Blood of their own.
"""

song2 = """
All I need
Is for you to come and rescue me
From this ardent fire pit
That I, in all my writhing, lit
You see the dragon that holds
My breathing in his tough and scaly folds
Well, why you waiting?
Come and slay him
So we can be happy
Draw your sword
Pray the lord
Don’t make you too sappy

Cause aoaoaoaoao
Long I bleed
Can’t you follow all the stains on me
Darling is it too much
To come and free the woman that you touch?
Oh, it’s a wretched affair
You, and me, and the dragon and his lair
Should I trust you?
The coward in you
Is begging to be broken
From the chains
That I reign
Thought I left you with a token

But aoaoaoaoao
Oh my sweet
Won’t you ride and prove your love to me
Walk my impossible line
While my dear dragon and I dine
And till you do your solemn duty
I shall lay with his beauty
So why you waiting
The sun is waning
The fire is flaying
All my skin
Head to shin
So go on and get slayin

Cause aoaoaoaoao
Oh my love, remember when we wept in snow?
And oh my love, if you left me where would I go?
The Forest strays so dark and deep before the shore
What’s more my love, for you I’d kill most any whore
Take from me, my deepest devotion
Take from me, my soul and all my notions
Take from me, the salt on my quiet skin
Take from me, all the voices from within
"""


song1 = """
You make me happy, I think
Calm like the sea
And yet, with roaring tides
A vigourius, unrelenting life force.
One that eats and devours and kills.
A carved seabed with untold thrills.
Thrills that betray and thrills that bury
Then rise again, in the light of the orb of night.
And there on the shore, the girl, weeping, starys.
And there the man shouts, till she returns to the fray
And fight they do, with all worldly sin
Till they win, and fly and eat it from within.
"""


### Choose a song

Decide which song you want to play with. And then insert the name (say songX) in the code cell below like this -

song = songX

In [ ]:
song = song7 

# Processing the song

Run the code cell below. It will do some initial calculations for you.

VM1 might not understand all the words. **For whatever words it doesn't understand, it'll ask you to rewrite it. And fill in its syllable count.**


This might take some time. I'm guessing about 2-5 minutes. Be nice to VM1 and wait up.

In [ ]:
df = construct(song)
df, rhymezone, consonant, assonance = prep_stats(df)

# Let's get dirty

Run the cell below and see what VM1 thinks about your song.

In [ ]:
say_hi()

babytooth this song is wonderful


### Song Analytics


In [ ]:
song_stats(df)

### Verse Analytics

This will take you through each verse. It'll describe some basic stuff.

Since it's a lot of data to scroll through, I've added input bars after every verse. So once you're done reading a verse, hit [enter].

In [ ]:
stats_per_verse(df)

### Line Analytics

Just like the verses above, you'll have to hit [enter] after every line.

In [ ]:
stats_per_line(df)

# Recommenders

For every line, it'll tell you: 
1.  How much should you adjust your syllable count by.
2.  A list of relevant rhyming synonyms. And for each synonym- its syllable count and the word it rhymes with


For example, if you had to adjust your syllable count by 2, then you would look for synonyms that are bigger than the current word. 

Similarly, if you had to adjust your syllable count by -2, then you would look for synonyms that are smaller than the current word.

### Perfect rhyme recommender

This basically finds relevant synonyms of every word in each line.

Then it checks if these synonyms rhyme with any words in the verse.
If it does, it reccomends them to you.

You must pick whatever makes sense to you.

In [ ]:
perfect_recco(df, rhymezone)

### Assonance recommender
Assonance is the repetition of vowel or diphthong sounds in one or more words found close together.

Again, it finds the synonyms for each word. 
Then it checks if these synonyms are an assonance to the words in the verse.
If they are, it reccomends them to you.

You must pick whatever makes sense to you.

In [ ]:
asso_recco(df, assonance)

### Consonance recommender

Consonance is the repetition of specific consonant sounds in close proximity.

Again, it finds the synonyms for each word. 
Then it checks if these synonyms are an consonance to the words in the verse.
If they are, it reccomends them to you.

You must pick whatever makes sense to you.

In [ ]:
conso_recco(df, consonant)

# Quick tools

These are quick tools where you must enter the specific stanza, line or word.

### Display song and syllables


In [ ]:
display_song(df)

### Get synonyms


In [ ]:
syn_tool(df)

### Get Perfect Rhymes

In [ ]:
tool_perf_rhyme(df)

### Get Assonance

Either enter the line number, or a word from the poem.

In [ ]:
tool_asso(df)

### Get Consonants

Either enter a line number or a word from the poem

In [ ]:
tool_conso(df)

### Get syllable count

In [ ]:
ans = input("Enter a word: ")
count_syllables(ans)

### Get syllables and their stress

In [ ]:
tool_stress_syll()